# Nuevas features

In [2]:
import pandas as pd

In [3]:
dataset_scl = pd.read_csv("../input_data/dataset_SCL.csv", parse_dates=True)
# Fecha-I
dataset_scl['Fecha-I'] = pd.to_datetime(dataset_scl['Fecha-I'])
dataset_scl['Fecha-O'] = pd.to_datetime(dataset_scl['Fecha-O'])
# Limpieza códigos
dataset_scl = dataset_scl[~pd.to_numeric(dataset_scl['Vlo-I'], errors='coerce').isnull()]
dataset_scl = dataset_scl[~pd.to_numeric(dataset_scl['Vlo-O'], errors='coerce').isnull()]

C:\Users\alexa\AppData\Local\Temp\ipykernel_6516\2238979313.py:1: DtypeWarning: Columns (1,6) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset_scl = pd.read_csv("../input_data/dataset_SCL.csv", parse_dates=True)


**Temporada_alta**: 1 si Fecha-I está entre 15-Dic y 3-Mar, o 15-Jul y 31-Jul, o 11-Sep y
30-Sep, 0 si no.

In [4]:
import datetime
datetime.datetime(2017, 3, 3)

datetime.datetime(2017, 3, 3, 0, 0)

In [5]:
import datetime

def temporada_alta(fecha):
    if fecha <= datetime.datetime(2017, 3, 3):
        return 1
    elif fecha >= datetime.datetime(2017, 12, 15):
        return 1
    elif datetime.datetime(2017, 7, 15) <= fecha <= datetime.datetime(2017, 7, 31):
        return 1
    elif datetime.datetime(2017, 9, 11) <= fecha <= datetime.datetime(2017, 9, 30):
        return 1
    else:
        return 0

In [6]:
dataset_scl['temporada_alta'] = dataset_scl['Fecha-I'].map(lambda x: temporada_alta(x))

**dif_min**: diferencia en minutos entre Fecha-O y Fecha-I.

In [7]:
dataset_scl['dif_min'] = (dataset_scl['Fecha-O'] - dataset_scl['Fecha-I']).dt.total_seconds()/60

**atraso_15**: 1 si dif_min > 15, 0 si no.

In [8]:
import numpy as np

dataset_scl['atraso_15'] = np.where(dataset_scl['dif_min'] > 15, 1, 0)

**periodo_dia**: mañana (entre 5:00 y 11:59), tarde (entre 12:00 y 18:59) y noche
(entre 19:00 y 4:59), en base a Fecha-I 

In [9]:
def periodo_dia(x):
    if x.time() < datetime.time(5):
        return 'noche'
    elif x.time() < datetime.time(12):
        return 'mañana'
    elif x.time() < datetime.time(19):
        return 'tarde'
    else:
        return 'noche'

In [10]:
dataset_scl['periodo_dia'] = dataset_scl['Fecha-I'].map(lambda x: periodo_dia(x))

In [11]:
dataset_scl.loc[:,['Fecha-I', 'periodo_dia']].sample(n=10)

,Fecha-I,periodo_dia
16136,2017-03-10 20:10:00,noche
39147,2017-08-16 12:25:00,tarde
11153,2017-02-06 08:50:00,mañana
13966,2017-03-01 13:30:00,tarde
4357,2017-01-18 22:01:00,noche
30972,2017-06-21 14:10:00,tarde
42341,2017-08-20 20:00:00,noche
12913,2017-03-05 08:10:00,mañana
26146,2017-05-28 14:50:00,tarde
20064,2017-04-04 14:55:00,tarde


Hora del día: Es posible que exista una relación entre la hora del vuelo y los atrasos

In [12]:
# Hora del viaje
dataset_scl['hora'] = pd.to_datetime(dataset_scl['Fecha-I'].astype(str)).dt.hour

### Cantidad de vuelos por día

Tengo la intuición de que los días con una cantidad alta de vuelos programados, tienden a tener una mayor probabilidad de atraso.

Pero construir una feature de cantidad de vuelos por día no es muy precisa, si un vuelo está cerca de las 01 horas, es posible que se atrase por los vuelos atrasados del día anterior (un par de horas antes).

Creo que buenas features para representar este fenomeno serían las siguientes: n° vuelos programados en las últimas 12 horas y n° vuelos programados en las siguientes 12 horas.
Esto nos ayudará a representar la cantidad de carga asociada a cierto periodo en el aeropuerto, atrasando la logistica de los vuelos.

In [38]:
fechas_vuelos = dataset_scl['Fecha-I']

fecha = dataset_scl['Fecha-I'].iloc[30000]

(
    (fecha - pd.DateOffset(hours=12) < fechas_vuelos) &
    (fecha > fechas_vuelos)
).sum()

96

In [43]:
fechas_vuelos = dataset_scl['Fecha-I']

fecha = dataset_scl['Fecha-I'].iloc[30000]

(
    (fecha + pd.DateOffset(hours=12) > fechas_vuelos) &
    (fecha < fechas_vuelos)
).sum()

62

In [59]:
fechas_vuelos = dataset_scl['Fecha-I']

def v_last_12(fecha, hours_delta):
    v_last_12 = (
        (fecha - pd.DateOffset(hours=hours_delta) < fechas_vuelos) &
        (fecha > fechas_vuelos)
    ).sum()
    return v_last_12

def v_next_12(fecha, hours_delta):
    v_next_12 = (
        (fecha + pd.DateOffset(hours=hours_delta) > fechas_vuelos) &
        (fecha < fechas_vuelos)
    ).sum()
    return v_next_12

In [46]:
dataset_scl['v_last_12'] = dataset_scl['Fecha-I'].map(lambda x: v_last_12(x, 12))
dataset_scl['v_next_12'] = dataset_scl['Fecha-I'].map(lambda x: v_next_12(x, 12))

In [60]:
dataset_scl['v_last_6'] = dataset_scl['Fecha-I'].map(lambda x: v_last_12(x, 6))
dataset_scl['v_next_6'] = dataset_scl['Fecha-I'].map(lambda x: v_next_12(x, 6))

In [61]:
dataset_scl['v_last_3'] = dataset_scl['Fecha-I'].map(lambda x: v_last_12(x, 3))
dataset_scl['v_next_3'] = dataset_scl['Fecha-I'].map(lambda x: v_next_12(x, 3))

In [63]:
dataset_scl.query("MES == 4 & DIA == 15").head(5)

,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,...,dif_min,atraso_15,periodo_dia,hora,v_last_12,v_next_12,v_last_6,v_next_6,v_last_3,v_next_3
17162,2017-04-15 21:00:00,912,SCEL,KMIA,AAL,2017-04-15 21:00:00,912,SCEL,KMIA,AAL,...,0.0,0,noche,21,70,54,31,17,12,10
17190,2017-04-15 21:30:00,940,SCEL,KDFW,AAL,2017-04-15 21:36:00,940,SCEL,KDFW,AAL,...,6.0,0,noche,21,66,60,29,16,11,8
17216,2017-04-15 11:35:00,92,SCEL,SAEZ,ACA,2017-04-15 11:49:00,92,SCEL,SAEZ,ACA,...,14.0,0,mañana,11,61,62,48,40,23,18
17239,2017-04-15 19:35:00,93,SCEL,CYYZ,ACA,2017-04-15 20:06:00,93,SCEL,CYYZ,ACA,...,31.0,1,noche,19,83,44,36,22,15,11
17266,2017-04-15 13:15:00,401,SCEL,LFPG,AFR,2017-04-15 13:14:00,401,SCEL,LFPG,AFR,...,-1.0,0,tarde,13,68,55,46,35,18,21


In [64]:
dataset_scl.query("MES == 2 & DIA == 15").head(5)

,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,...,dif_min,atraso_15,periodo_dia,hora,v_last_12,v_next_12,v_last_6,v_next_6,v_last_3,v_next_3
6113,2017-02-15 23:30:00,226,SCEL,KMIA,AAL,2017-02-15 23:18:00,226,SCEL,KMIA,AAL,...,-12.0,0,noche,23,121,81,55,19,23,12
6131,2017-02-15 22:00:00,912,SCEL,KMIA,AAL,2017-02-15 21:57:00,912,SCEL,KMIA,AAL,...,-3.0,0,noche,22,117,82,59,26,28,18
6158,2017-02-15 22:30:00,940,SCEL,KDFW,AAL,2017-02-15 22:28:00,940,SCEL,KDFW,AAL,...,-2.0,0,noche,22,119,77,57,19,27,13
6183,2017-02-15 13:35:00,92,SCEL,SAEZ,ACA,2017-02-15 14:45:00,92,SCEL,SAEZ,ACA,...,70.0,1,tarde,13,93,106,58,63,23,33
6207,2017-02-15 21:10:00,93,SCEL,CYYZ,ACA,2017-02-15 21:28:00,93,SCEL,CYYZ,ACA,...,18.0,1,noche,21,121,80,61,30,30,21


Podemos ver como aumenta la carga operacional del aeropuerto en Febrero, mientras que en Abril la carga es mucho menor. Estas features pueden ser muy útiles, son números que se pueden correlacionar positivamente con la probabilidad de que un vuelo se atrase

In [65]:
dataset_scl.to_csv('../clean_data/synthetic_features.csv', index=False)

In [66]:
synthetic = pd.read_csv('../clean_data/synthetic_features.csv', parse_dates=['Fecha-I', 'Fecha-O'])
print(synthetic.dtypes)
synthetic

Fecha-I           datetime64[ns]
Vlo-I                      int64
Ori-I                     object
Des-I                     object
Emp-I                     object
Fecha-O           datetime64[ns]
Vlo-O                    float64
Ori-O                     object
Des-O                     object
Emp-O                     object
DIA                        int64
MES                        int64
AÑO                        int64
DIANOM                    object
TIPOVUELO                 object
OPERA                     object
SIGLAORI                  object
SIGLADES                  object
temporada_alta             int64
dif_min                  float64
atraso_15                  int64
periodo_dia               object
hora                       int64
v_last_12                  int64
v_next_12                  int64
v_last_6                   int64
v_next_6                   int64
v_last_3                   int64
v_next_3                   int64
dtype: object


,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,...,dif_min,atraso_15,periodo_dia,hora,v_last_12,v_next_12,v_last_6,v_next_6,v_last_3,v_next_3
0,2017-01-01 23:30:00,226,SCEL,KMIA,AAL,2017-01-01 23:33:00,226.0,SCEL,KMIA,AAL,...,3.0,0,noche,23,106,89,46,25,18,15
1,2017-01-02 23:30:00,226,SCEL,KMIA,AAL,2017-01-02 23:39:00,226.0,SCEL,KMIA,AAL,...,9.0,0,noche,23,110,85,51,21,20,12
2,2017-01-03 23:30:00,226,SCEL,KMIA,AAL,2017-01-03 23:39:00,226.0,SCEL,KMIA,AAL,...,9.0,0,noche,23,108,83,49,22,19,15
3,2017-01-04 23:30:00,226,SCEL,KMIA,AAL,2017-01-04 23:33:00,226.0,SCEL,KMIA,AAL,...,3.0,0,noche,23,110,80,46,21,17,13
4,2017-01-05 23:30:00,226,SCEL,KMIA,AAL,2017-01-05 23:28:00,226.0,SCEL,KMIA,AAL,...,-2.0,0,noche,23,115,83,54,21,27,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68180,2017-12-22 14:55:00,400,SCEL,SPJC,JAT,2017-12-22 15:41:00,400.0,SCEL,SPJC,JAT,...,46.0,1,tarde,14,121,104,63,70,37,33
68181,2017-12-25 14:55:00,400,SCEL,SPJC,JAT,2017-12-25 15:11:00,400.0,SCEL,SPJC,JAT,...,16.0,1,tarde,14,111,89,63,63,35,32
68182,2017-12-27 14:55:00,400,SCEL,SPJC,JAT,2017-12-27 15:35:00,400.0,SCEL,SPJC,JAT,...,40.0,1,tarde,14,119,96,62,64,38,33
68183,2017-12-29 14:55:00,400,SCEL,SPJC,JAT,2017-12-29 15:08:00,400.0,SCEL,SPJC,JAT,...,13.0,0,tarde,14,117,97,63,65,38,31
